In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import re

Sortierung und Anpassung der Adressbezeichnungen für ein mögliches Verbinden der Daten:

In [ ]:
#Trennung der unterschiedlichen Straßen in einem Feature, getrennt durch ein Simikolon:

# Eingabe- und Ausgabepfade
input_shapefile = 
output_shapefile = 

# Shape-Datei einlesen
gdf = gpd.read_file(input_shapefile)

# Überprüfen, ob die Spalte "LAGEBEZTXT" existiert
if "LAGEBEZTXT" not in gdf.columns:
    raise ValueError("Die Spalte 'LAGEBEZTXT' ist in der Eingabedatei nicht vorhanden.")

# Liste für die neuen Datenzeilen
new_rows = []

# Iteriere durch die Geodaten
for _, row in gdf.iterrows():
    lagebez = row["LAGEBEZTXT"]  # Adressspalte
    geometry = row.geometry  # Geometrie
    other_columns = row.drop(["LAGEBEZTXT", "geometry"])  # Restliche Spalten

    if lagebez:
        # Adressen durch ";" trennen und bereinigen
        addresses = [addr.strip() for addr in lagebez.split(";") if addr.strip()]

        for address in addresses:
            if " " in address:
                # Trenne Straße und Hausnummern
                street, house_numbers = address.rsplit(" ", 1)

                # Hausnummern durch "," trennen und bereinigen
                house_numbers_list = [hn.strip() for hn in house_numbers.split(",") if hn.strip()]

                for house_number in house_numbers_list:
                    # Kombiniere Straße mit einzelner Hausnummer
                    new_address = f"{street} {house_number}"
                    new_row = other_columns.to_dict()
                    new_row["LAGEBEZTXT"] = new_address
                    new_row["geometry"] = geometry
                    new_rows.append(new_row)
            else:
                # Adresse ohne Hausnummer übernehmen
                new_row = other_columns.to_dict()
                new_row["LAGEBEZTXT"] = address
                new_row["geometry"] = geometry
                new_rows.append(new_row)

# Neues GeoDataFrame erstellen
new_gdf = gpd.GeoDataFrame(new_rows, geometry="geometry", crs=gdf.crs)

# Neue Shape-Datei speichern
new_gdf.to_file(output_shapefile, driver="ESRI Shapefile")

print("Adressen erfolgreich gesplittet und neue Shape-Datei erstellt.")


Adressen erfolgreich gesplittet und neue Shape-Datei erstellt.


In [ ]:
#Trennung am Komma und erneutes Zusammenfügen der Adressen mit multiplen Hausnummern für das gleiche Gebäude:

# Eingabe- und Ausgabepfade
input_shapefile = 
output_shapefile = 
# Shape-Datei einlesen
gdf = gpd.read_file(input_shapefile)

if "LAGEBEZTXT" not in gdf.columns:
    raise ValueError("Die Spalte 'LAGEBEZTXT' ist in der Eingabedatei nicht vorhanden.")

# Liste für die neuen Datenzeilen
new_rows = []

# Iteriere durch die Zeilen der Shape-Datei
for _, row in gdf.iterrows():
    lagebez = row["LAGEBEZTXT"]
    geometry = row.geometry
    other_columns = row.drop(["LAGEBEZTXT", "geometry"])

    if lagebez:
        # Adressentrennung mithilfe von isdigit()
        split_index = None
        for i, char in enumerate(lagebez):
            if char.isdigit():  # Prüfe, ob das Zeichen eine Ziffer ist
                split_index = i
                break

        # Trenne die Basisadresse (Straßenname) und die Hausnummern
        if split_index is not None:
            street = lagebez[:split_index].strip()  # Alles vor der ersten Ziffer
            house_numbers = lagebez[split_index:].strip()  # Alles ab der ersten Ziffer
        else:
            # Wenn keine Ziffer gefunden wird
            street = lagebez.strip()
            house_numbers = ""

        # Hausnummern durch ',' trennen
        house_numbers_list = [hn.strip() for hn in house_numbers.split(",") if hn.strip()]

        for house_number in house_numbers_list:
            new_address = f"{street} {house_number}"
            new_row = other_columns.to_dict()
            new_row["LAGEBEZTXT"] = new_address
            new_row["geometry"] = geometry
            new_rows.append(new_row)

# Neues GeoDataFrame erstellen
new_gdf = gpd.GeoDataFrame(new_rows, geometry="geometry", crs=gdf.crs)

# Neue Shape-Datei speichern
new_gdf.to_file(output_shapefile, driver="ESRI Shapefile")

print("Die Shape-Datei wurde erfolgreich angepasst und gespeichert.")


Die Shape-Datei wurde erfolgreich angepasst und gespeichert.


Aggregation der Sinusdaten auf Baublöcke nach dem dominanten Milieu

Eingabedaten als Polygone:

In [ ]:
# Aggregieren der gebäudebezogenen SINUS-Daten bezüglich der Anzahl an Haushalten pro Milieu in einem Zielpolygon (Baublöcke oder Lambert-Raster)

# Pfade zu den Dateien
input_polygons_with_data_path =   # Eingabe-Polygone mit Haushalts- und Milieudaten
target_polygons_path =   # Ziel-Polygone
output_aggregated_path =   # Ausgabe-Shape-Datei

# Feldnamen
household_field = "MB_HA_BA_A"  # Anzahl der Haushalte
milieu_field = "MB_HA_SI_K"  # Dominantes Milieu

# Shape-Dateien einlesen
print("Shape-Dateien werden eingelesen...")
input_polygons_with_data = gpd.read_file(input_polygons_with_data_path)
target_polygons = gpd.read_file(target_polygons_path)

# Berechne die Mittelpunkte der Eingabepolygone
print("Mittelpunkte der Eingabepolygone werden berechnet...")
input_polygons_with_data["centroid"] = input_polygons_with_data.geometry.centroid
input_centroids = input_polygons_with_data.set_geometry("centroid")

# Räumliche Verknüpfung: Mittelpunkte mit Zielpolygonen verknüpfen
print("Räumliche Verknüpfung wird durchgeführt...")
joined = gpd.sjoin(input_centroids, target_polygons, how="inner", predicate="within")

# Aggregation: Haushalte pro dominantem Milieu und Polygon
print("Aggregation wird durchgeführt...")
aggregation = (
    joined.groupby(["index_right", milieu_field])[household_field]
    .sum()
    .reset_index()
    .rename(columns={"index_right": "Polygon_ID", household_field: "Total_Households"})
)

# Aggregierte Daten mit den Zielpolygonen verknüpfen
print("Aggregierte Daten werden mit Zielpolygonen verknüpft...")
target_polygons["Polygon_ID"] = target_polygons.index  # ID-Feld hinzufügen
aggregated_polygons = target_polygons.merge(aggregation, on="Polygon_ID", how="left")

# Ergebnis speichern
print(f"Ergebnis wird gespeichert unter: {output_aggregated_path}")
aggregated_polygons.to_file(output_aggregated_path, driver="ESRI Shapefile")

print("Aggregation abgeschlossen.")


Shape-Dateien werden eingelesen...
Mittelpunkte der Eingabepolygone werden berechnet...
Räumliche Verknüpfung wird durchgeführt...
Aggregation wird durchgeführt...
Aggregierte Daten werden mit Zielpolygonen verknüpft...
Ergebnis wird gespeichert unter: C:\Users\ygerling\WWNW\Geodaten\SINUS2024\Wohngeb_HB\Ergebnis aggregation\Wohngeb_HB_AdressUpdate_SinusJoin_aggregiert.shp


C:\Users\ygerling\AppData\Local\Temp\ipykernel_45392\549919288.py:42: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  aggregated_polygons.to_file(output_aggregated_path, driver="ESRI Shapefile")


Aggregation abgeschlossen.


Eingabedaten als Punkte:

In [ ]:
# Aggregieren der punktbezogenen SINUS-Daten bezüglich der Anzahl an Haushalten pro Milieu in einem Zielpolygon (Baublöcke oder Lambert-Raster)

# Pfade zu den Dateien
input_points_with_data_path =   # Eingabe-Punkte mit Haushalts- und Milieudaten
target_polygons_path =   # Ziel-Polygone
output_aggregated_path =   # Ausgabe-Shape-Datei

# Feldnamen
household_field = "MB_HA_BA_A"  # Anzahl der Haushalte
milieu_field = "MB_HA_SI_K"  # Dominantes Milieu

# Shape-Dateien einlesen
print("Shape-Dateien werden eingelesen...")
input_points_with_data = gpd.read_file(input_points_with_data_path)
target_polygons = gpd.read_file(target_polygons_path)

# Räumliche Verknüpfung: Punkte mit Zielpolygonen verknüpfen
print("Räumliche Verknüpfung wird durchgeführt...")
joined = gpd.sjoin(input_points_with_data, target_polygons, how="inner", predicate="within")

# Aggregation: Haushalte pro dominantem Milieu und Polygon
print("Aggregation wird durchgeführt...")
aggregation = (
    joined.groupby(["index_right", milieu_field])[household_field]
    .sum()
    .reset_index()
    .rename(columns={"index_right": "Polygon_ID", household_field: "Total_Households"})
)

# Aggregierte Daten mit den Zielpolygonen verknüpfen
print("Aggregierte Daten werden mit Zielpolygonen verknüpft...")
target_polygons["Polygon_ID"] = target_polygons.index  # ID-Feld hinzufügen
aggregated_polygons = target_polygons.merge(aggregation, on="Polygon_ID", how="left")

# Ergebnis speichern
print(f"Ergebnis wird gespeichert unter: {output_aggregated_path}")
aggregated_polygons.to_file(output_aggregated_path, driver="ESRI Shapefile")

print("Aggregation abgeschlossen.")


Shape-Dateien werden eingelesen...
Räumliche Verknüpfung wird durchgeführt...
Aggregation wird durchgeführt...
Aggregierte Daten werden mit Zielpolygonen verknüpft...
Ergebnis wird gespeichert unter: C:\Users\ygerling\WWNW\Geodaten\SINUS2024\Ergebnisse 25\AX_GeoreferenzierteGebaeudeadresse_Wohngeb_2025_BB_aggregiert.shp


C:\Users\ygerling\AppData\Local\Temp\ipykernel_28068\1767452773.py:37: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  aggregated_polygons.to_file(output_aggregated_path, driver="ESRI Shapefile")


Aggregation abgeschlossen.


Zusammenfassung der Baublöcke unter der Baublock ID:

In [ ]:
# Definieren des dominanten Milieus pro Zielpolygon

# Shape-Datei einlesen (ersetze 'datei.shp' durch den tatsächlichen Pfad)
file_path = 
gdf = gpd.read_file(file_path)

gdf = gpd.read_file(file_path)

# Relevante Spalten auswählen
df = gdf[['BB_Nr', 'MB_HA_SI_K', 'Total_Hous', 'geometry']]

# Sicherstellen, dass 'Total_Hous' numerisch ist
df['Total_Hous'] = pd.to_numeric(df['Total_Hous'], errors='coerce').fillna(0)

# Dominantes Milieu für jeden Baublock bestimmen
df_dominant = df.loc[df.groupby('BB_Nr')['Total_Hous'].idxmax(), ['BB_Nr', 'MB_HA_SI_K']]
df_dominant = df_dominant.rename(columns={'MB_HA_SI_K': 'Dominant_Milieu'})

# Aggregation der Daten nach Baublock-ID
df_agg = df.groupby('BB_Nr').agg({
    'MB_HA_SI_K': lambda x: ', '.join(x.astype(str)),  # Alle Milieus in einer Zeile zusammenfassen
    'Total_Hous': lambda x: ', '.join(x.astype(str)),  # Haushaltszahlen entsprechend auflisten
    'geometry': 'first'  # Eine Geometrie für den Baublock übernehmen
}).reset_index()

# Das dominante Milieu hinzufügen
df_agg = df_agg.merge(df_dominant, on='BB_Nr', how='left')

# In eine neue Shape-Datei speichern
output_file = 
df_agg_gdf = gpd.GeoDataFrame(df_agg, geometry='geometry')
df_agg_gdf.to_file(output_file, driver='ESRI Shapefile')

# Ergebnis ausgeben (ersten 5 Zeilen)
print(df_agg.head())



c:\Users\ygerling\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\ygerling\AppData\Local\Temp\ipykernel_28068\243809689.py:32: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_agg_gdf.to_file(output_file, driver='ESRI Shapefile')


      BB_Nr MB_HA_SI_K Total_Hous  \
0  111001.0       None        0.0   
1  111002.0       None        0.0   
2  111003.0       None        0.0   
3  111004.0     04_EPE        2.0   
4  111005.0       None        0.0   

                                            geometry Dominant_Milieu  
0  POLYGON ((485993.727 5881383.175, 486002.187 5...            None  
1  POLYGON ((486350.774 5881495.173, 486345.437 5...            None  
2  POLYGON ((486547.822 5881501.250, 486559.001 5...            None  
3  POLYGON ((486933.524 5881216.109, 486948.888 5...          04_EPE  
4  POLYGON ((487352.494 5881058.773, 487339.400 5...            None  
